In [1]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random
from sklearn.model_selection import GroupKFold
# custom imports
from multiprocessing import Pool  # Multiprocess Runs

warnings.filterwarnings('ignore')

- 全部采用tweedie_variance_power=1.25，score=0.48332
- STORES_IDS = ['CA_1', 'CA_2', 'CA_3', 'WI_2'] # tweedie_variance_power=1.05, 其他商店1.25。（score=0.47886）
- STORES_IDS = ['CA_1', 'CA_2', 'CA_3', 'WI_2'] # tweedie_variance_power=1.05, ['TX_1', 'TX_2', 'TX_3']1.1,其他商店1.25。（score=0.47566）
- STORES_IDS = ['CA_1', 'CA_2', 'CA_3', 'WI_2'] # tweedie_variance_power=1.05, ['TX_1', 'TX_2', 'TX_3', 'WI_3']1.1,['CA_4', 'WI_1']1.25。（score=0.47437）

-         if store_id in ['CA_3', 'WI_2']:
            lgb_params['tweedie_variance_power'] = 1.05
        elif store_id in ['WI_3']:
            lgb_params['tweedie_variance_power'] = 1.07
        elif store_id in ['CA_1', 'TX_1', 'TX_2']:
            lgb_params['tweedie_variance_power'] = 1.10
        elif store_id in ['CA_2', 'TX_3']:
            lgb_params['tweedie_variance_power'] = 1.15
        else:
            lgb_params['tweedie_variance_power'] = 1.25
（score=0.47523）

-         if store_id in ['CA_1', 'CA_2', 'CA_3', 'WI_2']:
            lgb_params['tweedie_variance_power'] = 1.05
        elif store_id in ['TX_1', 'TX_2', 'TX_3', 'WI_3']:
            lgb_params['tweedie_variance_power'] = 1.10
        else:
            lgb_params['tweedie_variance_power'] = 1.20
            
        if store_id in ['CA_3']:
            lgb_params['n_estimators'] = 1800
        else:
            lgb_params['n_estimators'] = 1400
（score=0.47421）


-         if store_id in ['CA_1', 'CA_2', 'CA_3', 'WI_2']:
            lgb_params['tweedie_variance_power'] = 1.05
        elif store_id in ['TX_1', 'TX_2', 'TX_3', 'WI_3']:
            lgb_params['tweedie_variance_power'] = 1.10
        else:
            lgb_params['tweedie_variance_power'] = 1.20
            
        if store_id in ['CA_2', 'CA_3']:
            lgb_params['n_estimators'] = 1800
        else:
            lgb_params['n_estimators'] = 1400
            
（score=0.47427）

In [2]:
########################### Helpers
#################################################################################
## Seeder
# :seed to make all processes deterministic     # type: int
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)


## Multiprocess Runs
def df_parallelize_run(func, t_split):
    num_cores = np.min([N_CORES, len(t_split)])
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, t_split), axis=1)
    pool.close()
    pool.join()
    return df

In [3]:
########################### Helper to load data by store ID
#################################################################################
# Read data
def get_data_by_store(store):

    # Read and contact basic feature
    df = pd.concat([
        pd.read_pickle(BASE),
        pd.read_pickle(PRICE).iloc[:, 2:],
        pd.read_pickle(CALENDAR).iloc[:, 2:]
    ],
                   axis=1)

    # Leave only relevant store
    df = df[df['store_id'] == store]

    # With memory limits we have to read
    # lags and mean encoding features
    # separately and drop items that we don't need.
    # As our Features Grids are aligned
    # we can use index to keep only necessary rows
    # Alignment is good for us as concat uses less memory than merge.
    df2 = pd.read_pickle(MEAN_ENC)[mean_features]
    df2 = df2[df2.index.isin(df.index)]

    df3 = pd.read_pickle(LAGS).iloc[:, 3:]
    df3 = df3[df3.index.isin(df.index)]

    df = pd.concat([df, df2], axis=1)
    del df2  # to not reach memory limit

    df = pd.concat([df, df3], axis=1)
    del df3  # to not reach memory limit

    # Create features list
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id', 'd', TARGET] + features]

    # Skipping first n rows
    df = df[df['d'] >= START_TRAIN].reset_index(drop=True)

    return df, features


# Recombine Test set after training
def get_base_test():
    base_test = pd.DataFrame()

    for store_id in STORES_IDS:
        temp_df = pd.read_pickle(AUX_MODELS + 'test_' + store_id + '.pkl')
        temp_df['store_id'] = store_id
        base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)

    return base_test


########################### Helper to make dynamic rolling lags
#################################################################################
def make_lag(LAG_DAY):
    lag_df = base_test[['id', 'd', TARGET]]
    col_name = 'sales_lag_' + str(LAG_DAY)
    lag_df[col_name] = lag_df.groupby([
        'id'
    ])[TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
    return lag_df[[col_name]]


def make_lag_roll(LAG_DAY):
    shift_day = LAG_DAY[0]
    roll_wind = LAG_DAY[1]
    lag_df = base_test[['id', 'd', TARGET]]
    col_name = 'rolling_mean_tmp_' + str(shift_day) + '_' + str(roll_wind)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(
        lambda x: x.shift(shift_day).rolling(roll_wind).mean())
    return lag_df[[col_name]]

In [4]:
########################### Model params
#################################################################################
import lightgbm as lgb
lgb_params = {
    'boosting_type': 'gbdt',
    'objective': 'tweedie',
    'metric': 'rmse',
    'subsample': 0.5,
    'subsample_freq': 1,
    'learning_rate': 0.03,
    'num_leaves': 2**11 - 1,
    'min_data_in_leaf': 2**12 - 1,
    'feature_fraction': 0.5,
    'max_bin': 100,
    'n_estimators': 1400,
    'boost_from_average': False,
    'verbose': 1,
#     'device': 'gpu',
#     'gpu_platform_id': -1,
#     'gpu_device_id': -1
}

# Let's look closer on params

## 'boosting_type': 'gbdt'
# we have 'goss' option for faster training
# but it normally leads to underfit.
# Also there is good 'dart' mode
# but it takes forever to train
# and model performance depends
# a lot on random factor
# https://www.kaggle.com/c/home-credit-default-risk/discussion/60921

## 'objective': 'tweedie'
# Tweedie Gradient Boosting for Extremely
# Unbalanced Zero-inflated Data
# https://arxiv.org/pdf/1811.10192.pdf
# and many more articles about tweediie
#
# Strange (for me) but Tweedie is close in results
# to my own ugly loss.
# My advice here - make OWN LOSS function
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/140564
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/143070
# I think many of you already using it (after poisson kernel appeared)
# (kagglers are very good with "params" testing and tuning).
# Try to figure out why Tweedie works.
# probably it will show you new features options
# or data transformation (Target transformation?).

## 'tweedie_variance_power': 1.1
# default = 1.5
# set this closer to 2 to shift towards a Gamma distribution
# set this closer to 1 to shift towards a Poisson distribution
# my CV shows 1.1 is optimal
# but you can make your own choice

## 'metric': 'rmse'
# Doesn't mean anything to us
# as competition metric is different
# and we don't use early stoppings here.
# So rmse serves just for general
# model performance overview.
# Also we use "fake" evaluation set
# (as it makes part of the training set)
# so even general rmse score doesn't mean anything))
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/133834

## 'subsample': 0.5
# Serves to fight with overfit
# this will randomly select part of data without resampling
# Chosen by CV (my CV can be wrong!)
# Next kernel will be about CV

##'subsample_freq': 1
# frequency for bagging
# default value - seems ok

## 'learning_rate': 0.03
# Chosen by CV
# Smaller - longer training
# but there is an option to stop
# in "local minimum"
# Bigger - faster training
# but there is a chance to
# not find "global minimum" minimum

## 'num_leaves': 2**11-1
## 'min_data_in_leaf': 2**12-1
# Force model to use more features
# We need it to reduce "recursive"
# error impact.
# Also it leads to overfit
# that's why we use small
# 'max_bin': 100

## l1, l2 regularizations
# https://towardsdatascience.com/l1-and-l2-regularization-methods-ce25e7fc831c
# Good tiny explanation
# l2 can work with bigger num_leaves
# but my CV doesn't show boost

## 'n_estimators': 1400
# CV shows that there should be
# different values for each state/store.
# Current value was chosen
# for general purpose.
# As we don't use any early stopings
# careful to not overfit Public LB.

##'feature_fraction': 0.5
# LightGBM will randomly select
# part of features on each iteration (tree).
# We have maaaany features
# and many of them are "duplicates"
# and many just "noise"
# good values here - 0.5-0.7 (by CV)

## 'boost_from_average': False
# There is some "problem"
# to code boost_from_average for
# custom loss
# 'True' makes training faster
# BUT carefull use it
# https://github.com/microsoft/LightGBM/issues/1514
# not our case but good to know cons

In [5]:
########################### Vars
#################################################################################
SEED = 42  # We want all things
seed_everything(SEED)  # to be as deterministic
lgb_params['seed'] = SEED  # as possible
N_CORES = psutil.cpu_count()  # Available CPU cores

#LIMITS and const
TARGET = 'sales'  # Our target
START_TRAIN = 0  # We can skip some rows (Nans/faster training)
END_TRAIN = 1941  # End day of our train set
P_HORIZON = 28  # Prediction horizon
USE_AUX = False  # Use or not pretrained models

#FEATURES to remove
## These features lead to overfit
## or values not present in test set
remove_features = [
    'id', 'state_id', 'store_id', 'date', 'wm_yr_wk', 'd', TARGET
]
mean_features = [
    'enc_cat_id_mean', 'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std',
    'enc_item_id_mean', 'enc_item_id_std'
]

#PATHS for Features
ORIGINAL = '../data/'
BASE = '../data/output/grid_part_1.pkl'
PRICE = '../data/output/grid_part_2.pkl'
CALENDAR = '../data/output/grid_part_3.pkl'
LAGS = '../data/output/lags_df_28.pkl'
MEAN_ENC = '../data/output/mean_encoding_df.pkl'

# AUX(pretrained) Models paths
AUX_MODELS = '../data/output/'

#STORES ids
STORES_IDS = pd.read_csv(ORIGINAL+'sales_train_evaluation.csv')['store_id']
STORES_IDS = list(STORES_IDS.unique())
# STORES_IDS = [
#     'CA_1'
# ]  # It takes around 5hrs to train for each store. Please run multiple kernels for each store.

#FOLDS
CV_FOLDS = [0, 1, 2]

#SPLITS for lags creation
SHIFT_DAY = 28
N_LAGS = 15
LAGS_SPLIT = [col for col in range(SHIFT_DAY, SHIFT_DAY + N_LAGS)]
ROLS_SPLIT = []
for i in [1, 7, 14]:
    for j in [7, 14, 30, 60]:
        ROLS_SPLIT.append([i, j])

In [6]:
########################### Aux Models
# If you don't want to wait hours and hours
# to have result you can train each store
# in separate kernel and then just join result.

# If we want to use pretrained models we can
## skip training
## (in our case do dummy training
##  to show that we are good with memory
##  and you can safely use this (all kernel) code)
# if USE_AUX:
#     lgb_params['n_estimators'] = 2

# Here is some 'logs' that can compare
#Train CA_1
#[100]	valid_0's rmse: 2.02289
#[200]	valid_0's rmse: 2.0017
#[300]	valid_0's rmse: 1.99239
#[400]	valid_0's rmse: 1.98471
#[500]	valid_0's rmse: 1.97923
#[600]	valid_0's rmse: 1.97284
#[700]	valid_0's rmse: 1.96763
#[800]	valid_0's rmse: 1.9624
#[900]	valid_0's rmse: 1.95673
#[1000]	valid_0's rmse: 1.95201
#[1100]	valid_0's rmse: 1.9476
#[1200]	valid_0's rmse: 1.9434
#[1300]	valid_0's rmse: 1.9392
#[1400]	valid_0's rmse: 1.93446

#Train CA_2
#[100]	valid_0's rmse: 1.88949
#[200]	valid_0's rmse: 1.84767
#[300]	valid_0's rmse: 1.83653
#[400]	valid_0's rmse: 1.82909
#[500]	valid_0's rmse: 1.82265
#[600]	valid_0's rmse: 1.81725
#[700]	valid_0's rmse: 1.81252
#[800]	valid_0's rmse: 1.80736
#[900]	valid_0's rmse: 1.80242
#[1000]	valid_0's rmse: 1.79821
#[1100]	valid_0's rmse: 1.794
#[1200]	valid_0's rmse: 1.78973
#[1300]	valid_0's rmse: 1.78552
#[1400]	valid_0's rmse: 1.78158

In [7]:
STORES_IDS

['CA_1',
 'CA_2',
 'CA_3',
 'CA_4',
 'TX_1',
 'TX_2',
 'TX_3',
 'WI_1',
 'WI_2',
 'WI_3']

In [8]:
########################### Train Models
#################################################################################

for store_id in STORES_IDS:
    print('Train', store_id)

    # Get grid for current store
    grid_df, features_columns = get_data_by_store(store_id)

    # Masks for
    # Train (All data less than 1941)
    # "evaluation" (Last 28 days - not real validatio set)
    # Test (All data greater than 1941 day,
    #       with some gap for recursive features)
    train_mask = grid_df['d'] <= END_TRAIN
    preds_mask = grid_df['d'] > (END_TRAIN - 100)

    ## Initiating our GroupKFold
    folds = GroupKFold(n_splits=3)

    # get subgroups for each week, year pair
    grid_df['groups'] = grid_df['tm_w'].astype(
        str) + '_' + grid_df['tm_y'].astype(str)
    split_groups = grid_df[train_mask]['groups']

    # Main Data
    X, y = grid_df[train_mask][features_columns], grid_df[train_mask][TARGET]

    # Saving part of the dataset for later predictions
    # Removing features that we need to calculate recursively
    grid_df = grid_df[preds_mask].reset_index(drop=True)
    keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
    grid_df = grid_df[keep_cols]
    grid_df.to_pickle('../data/output/test_' + store_id + '.pkl')
    del grid_df

    # Launch seeder again to make lgb training 100% deterministic
    # with each "code line" np.random "evolves"
    # so we need (may want) to "reset" it

    for fold_, (trn_idx,
                val_idx) in enumerate(folds.split(X, y, groups=split_groups)):
        print('Fold:', fold_)
        print(len(trn_idx), len(val_idx))
        tr_x, tr_y = X.iloc[trn_idx, :], y[trn_idx]
        v_X, v_y = X.iloc[val_idx, :], y[val_idx]
        train_data = lgb.Dataset(tr_x, label=tr_y)
        valid_data = lgb.Dataset(v_X, label=v_y)      

        seed_everything(SEED)
        if store_id in ['CA_1', 'CA_2', 'CA_3', 'WI_2']:
            lgb_params['tweedie_variance_power'] = 1.05
        elif store_id in ['TX_1', 'TX_2', 'TX_3', 'WI_3']:
            lgb_params['tweedie_variance_power'] = 1.10
        else:
            lgb_params['tweedie_variance_power'] = 1.20
            
        if store_id in ['CA_2', 'CA_3']:
            lgb_params['n_estimators'] = 1800
        else:
            lgb_params['n_estimators'] = 1400
        
            
        estimator = lgb.train(lgb_params,
                              train_data,
                              valid_sets=[train_data, valid_data],
                              verbose_eval=100)
        # Save model - it's not real '.bin' but a pickle file
        # estimator = lgb.Booster(model_file='model.txt')
        # can only predict with the best iteration (or the saving iteration)
        # pickle.dump gives us more flexibility
        # like estimator.predict(TEST, num_iteration=100)
        # num_iteration - number of iteration want to predict with,
        # NULL or <= 0 means use best iteration
        model_name = 'lgb_model_' + store_id + '_' + str(fold_) + '.bin'
        pickle.dump(estimator, open(model_name, 'wb'))

        # Remove temporary files and objects
        # to free some hdd space and ram memory
        del train_data, valid_data, estimator
        gc.collect()

    # "Keep" models features for predictions
    MODEL_FEATURES = features_columns

Train CA_1
Fold: 0
3190193 1598074
[100]	training's rmse: 2.55329	valid_1's rmse: 2.63294
[200]	training's rmse: 2.46016	valid_1's rmse: 2.55576
[300]	training's rmse: 2.42652	valid_1's rmse: 2.53348
[400]	training's rmse: 2.40245	valid_1's rmse: 2.52023
[500]	training's rmse: 2.38385	valid_1's rmse: 2.51348
[600]	training's rmse: 2.36762	valid_1's rmse: 2.50802
[700]	training's rmse: 2.3534	valid_1's rmse: 2.50452
[800]	training's rmse: 2.34035	valid_1's rmse: 2.50187
[900]	training's rmse: 2.32902	valid_1's rmse: 2.4994
[1000]	training's rmse: 2.31761	valid_1's rmse: 2.49839
[1100]	training's rmse: 2.30717	valid_1's rmse: 2.49726
[1200]	training's rmse: 2.29686	valid_1's rmse: 2.49565
[1300]	training's rmse: 2.2876	valid_1's rmse: 2.49458
[1400]	training's rmse: 2.27874	valid_1's rmse: 2.49423
Fold: 1
3189915 1598352
[100]	training's rmse: 2.60636	valid_1's rmse: 2.50067
[200]	training's rmse: 2.51125	valid_1's rmse: 2.43441
[300]	training's rmse: 2.4718	valid_1's rmse: 2.41604
[400]

[1300]	training's rmse: 3.14132	valid_1's rmse: 3.46736
[1400]	training's rmse: 3.12846	valid_1's rmse: 3.46763
[1500]	training's rmse: 3.11624	valid_1's rmse: 3.46607
[1600]	training's rmse: 3.1042	valid_1's rmse: 3.46555
[1700]	training's rmse: 3.09292	valid_1's rmse: 3.46453
[1800]	training's rmse: 3.08249	valid_1's rmse: 3.46458
Train CA_4
Fold: 0
3099650 1552908
[100]	training's rmse: 1.53455	valid_1's rmse: 1.52982
[200]	training's rmse: 1.4969	valid_1's rmse: 1.50021
[300]	training's rmse: 1.48448	valid_1's rmse: 1.49237
[400]	training's rmse: 1.47684	valid_1's rmse: 1.489
[500]	training's rmse: 1.47024	valid_1's rmse: 1.48725
[600]	training's rmse: 1.46442	valid_1's rmse: 1.48609
[700]	training's rmse: 1.45964	valid_1's rmse: 1.48554
[800]	training's rmse: 1.45489	valid_1's rmse: 1.48489
[900]	training's rmse: 1.45071	valid_1's rmse: 1.48485
[1000]	training's rmse: 1.44678	valid_1's rmse: 1.4848
[1100]	training's rmse: 1.44273	valid_1's rmse: 1.48486
[1200]	training's rmse: 1.4

[1300]	training's rmse: 1.98534	valid_1's rmse: 2.10458
[1400]	training's rmse: 1.97758	valid_1's rmse: 2.10402
Fold: 1
3155716 1581451
[100]	training's rmse: 2.21334	valid_1's rmse: 2.27316
[200]	training's rmse: 2.11853	valid_1's rmse: 2.1875
[300]	training's rmse: 2.08408	valid_1's rmse: 2.16785
[400]	training's rmse: 2.06224	valid_1's rmse: 2.16029
[500]	training's rmse: 2.04377	valid_1's rmse: 2.15557
[600]	training's rmse: 2.02864	valid_1's rmse: 2.15247
[700]	training's rmse: 2.01545	valid_1's rmse: 2.15197
[800]	training's rmse: 2.0038	valid_1's rmse: 2.1508
[900]	training's rmse: 1.99292	valid_1's rmse: 2.15099
[1000]	training's rmse: 1.98301	valid_1's rmse: 2.15122
[1100]	training's rmse: 1.97275	valid_1's rmse: 2.152
[1200]	training's rmse: 1.96403	valid_1's rmse: 2.15185
[1300]	training's rmse: 1.95619	valid_1's rmse: 2.1526
[1400]	training's rmse: 1.94846	valid_1's rmse: 2.15321
Fold: 2
3156446 1580721
[100]	training's rmse: 2.25339	valid_1's rmse: 2.21986
[200]	training's

[500]	training's rmse: 2.25856	valid_1's rmse: 2.2794
[600]	training's rmse: 2.23828	valid_1's rmse: 2.27979
[700]	training's rmse: 2.22069	valid_1's rmse: 2.28413
[800]	training's rmse: 2.20513	valid_1's rmse: 2.28935
[900]	training's rmse: 2.19243	valid_1's rmse: 2.29252
[1000]	training's rmse: 2.18069	valid_1's rmse: 2.29642
[1100]	training's rmse: 2.16957	valid_1's rmse: 2.29833
[1200]	training's rmse: 2.15825	valid_1's rmse: 2.29911
[1300]	training's rmse: 2.14877	valid_1's rmse: 2.30033
[1400]	training's rmse: 2.13929	valid_1's rmse: 2.30071


In [9]:
########################### Predict
#################################################################################

for fold_ in CV_FOLDS:
    print("FOLD:", fold_)
    # Join back the Test dataset with
    # a small part of the training data
    # to make recursive features
    all_preds = pd.DataFrame()
    base_test = get_base_test()
    # Timer to measure predictions time
    main_time = time.time()

    # Loop over each prediction day
    # As rolling lags are the most timeconsuming
    # we will calculate it for whole day
    for PREDICT_DAY in range(1, 29):
        print('Predict | Day:', PREDICT_DAY)
        start_time = time.time()

        # Make temporary grid to calculate rolling lags
        grid_df = base_test.copy()
        grid_df = pd.concat(
            [grid_df, df_parallelize_run(make_lag_roll, ROLS_SPLIT)], axis=1)

        for store_id in STORES_IDS:

            # Read all our models and make predictions
            # for each day/store pairs
            # model_path = '../data/output/lgb_model_' + store_id + '_' + str(fold_) + '.bin'
            model_path = 'lgb_model_' + store_id + '_' + str(fold_) + '.bin'
            if USE_AUX:
                model_path = AUX_MODELS + model_path

            estimator = pickle.load(open(model_path, 'rb'))

            day_mask = base_test['d'] == (END_TRAIN + PREDICT_DAY)
            store_mask = base_test['store_id'] == store_id

            mask = (day_mask) & (store_mask)
            base_test[TARGET][mask] = estimator.predict(
                grid_df[mask][MODEL_FEATURES])

        # Make good column naming and add
        # to all_preds DataFrame
        temp_df = base_test[day_mask][['id', TARGET]]
        temp_df.columns = ['id', 'F' + str(PREDICT_DAY)]
        if 'id' in list(all_preds):
            all_preds = all_preds.merge(temp_df, on=['id'], how='left')
        else:
            all_preds = temp_df.copy()
        all_preds = all_preds.reset_index(drop=True)
        print('#' * 10,
              ' %0.2f min round |' % ((time.time() - start_time) / 60),
              ' %0.2f min total |' % ((time.time() - main_time) / 60),
              ' %0.2f day sales |' % (temp_df['F' + str(PREDICT_DAY)].sum()))
    all_preds.to_csv('all_preds_' + str(fold_) + '.csv', index=False)
    del temp_df, all_preds

FOLD: 0
Predict | Day: 1
##########  1.02 min round |  1.02 min total |  40009.04 day sales |
Predict | Day: 2
##########  1.03 min round |  2.06 min total |  37253.65 day sales |
Predict | Day: 3
##########  1.04 min round |  3.09 min total |  37162.95 day sales |
Predict | Day: 4
##########  1.03 min round |  4.13 min total |  37284.18 day sales |
Predict | Day: 5
##########  1.04 min round |  5.17 min total |  42455.90 day sales |
Predict | Day: 6
##########  1.06 min round |  6.23 min total |  50648.41 day sales |
Predict | Day: 7
##########  1.04 min round |  7.27 min total |  51590.05 day sales |
Predict | Day: 8
##########  1.02 min round |  8.29 min total |  45031.69 day sales |
Predict | Day: 9
##########  1.04 min round |  9.34 min total |  39050.77 day sales |
Predict | Day: 10
##########  1.05 min round |  10.39 min total |  44085.31 day sales |
Predict | Day: 11
##########  1.03 min round |  11.41 min total |  43994.04 day sales |
Predict | Day: 12
##########  1.03 min rou

In [10]:
# all_preds_0 = pd.read_csv(AUX_MODELS + 'all_preds_0' + '.csv')
# all_preds_1 = pd.read_csv(AUX_MODELS + 'all_preds_1' + '.csv')
# all_preds_2 = pd.read_csv(AUX_MODELS + 'all_preds_2' + '.csv')


all_preds_0 = pd.read_csv('all_preds_0' + '.csv')
all_preds_1 = pd.read_csv('all_preds_1' + '.csv')
all_preds_2 = pd.read_csv('all_preds_2' + '.csv')

In [11]:
# Create Dummy DataFrame to store predictions
final_all_preds = pd.DataFrame()
final_all_preds['id'] = all_preds_1['id']
for item in all_preds_1:
    if item != 'id':
        final_all_preds[item] = (all_preds_0[item] *
                                 (1 / 3)) + (all_preds_1[item] *
                                             (1 / 3)) + (all_preds_2[item] *
                                                         (1 / 3))
final_all_preds

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_evaluation,0.917347,0.795629,0.790688,0.820127,1.043406,1.281858,1.137568,1.028601,0.837616,...,0.984328,1.358549,1.222873,0.935263,0.855488,0.853315,0.889400,1.108676,1.262900,1.078745
1,HOBBIES_1_002_CA_1_evaluation,0.212920,0.184212,0.195551,0.187482,0.217524,0.290778,0.317841,0.235842,0.204217,...,0.267743,0.352177,0.381092,0.239658,0.231211,0.235798,0.249380,0.285975,0.367003,0.422879
2,HOBBIES_1_003_CA_1_evaluation,0.552828,0.482888,0.484098,0.520306,0.668682,0.787655,0.818071,0.576627,0.512080,...,0.625173,0.754726,0.750639,0.531554,0.466341,0.497379,0.485467,0.702056,0.769052,0.808288
3,HOBBIES_1_004_CA_1_evaluation,1.573001,1.337328,1.319863,1.389824,1.891648,2.557567,2.967362,1.973708,1.404140,...,1.919161,2.466931,2.992163,1.723306,1.385427,1.406102,1.443824,1.930967,2.629837,2.821440
4,HOBBIES_1_005_CA_1_evaluation,1.083407,1.000436,0.951119,1.006595,1.148831,1.378017,1.536918,1.185304,0.992965,...,1.273728,1.510603,1.525026,1.181781,1.070889,1.025914,1.078448,1.328520,1.573925,1.392928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,0.465869,0.443424,0.427163,0.446280,0.530074,0.538714,0.648807,0.528550,0.507078,...,0.587095,0.724158,0.841324,0.626166,0.622865,0.619947,0.479161,0.518167,0.586686,0.674935
30486,FOODS_3_824_WI_3_evaluation,0.260825,0.261031,0.231800,0.217816,0.221891,0.285150,0.301988,0.256184,0.268324,...,0.262176,0.381049,0.409017,0.301239,0.336960,0.360863,0.281665,0.252019,0.322181,0.311179
30487,FOODS_3_825_WI_3_evaluation,0.658070,0.533417,0.492366,0.484714,0.534814,0.679497,0.726037,0.651273,0.533949,...,0.785509,1.141680,1.271835,0.957491,1.055684,1.081464,0.732024,0.732628,0.792261,0.946860
30488,FOODS_3_826_WI_3_evaluation,0.999697,1.017071,0.979885,0.957912,1.060236,1.225254,1.154782,1.114489,1.009736,...,1.055475,1.497803,1.523842,1.165503,1.334722,1.298458,1.072061,1.079507,1.161220,1.190869


In [12]:
########################### Export
#################################################################################
# Reading competition sample submission and
# merging our predictions
# As we have predictions only for "_evaluation" data
# we need to do fillna() for "_evaluation" items
submission = pd.read_csv(ORIGINAL + 'sample_submission.csv')[['id']]
submission = submission.merge(final_all_preds, on=['id'], how='left').fillna(0)
submission.to_csv('submission_final' + '.csv', index=False)

In [13]:
# Summary

# Of course here is no magic at all.
# No "Novel" features and no brilliant ideas.
# We just carefully joined all
# our previous fe work and created a model.

# Also!
# In my opinion this strategy is a "dead end".
# Overfits a lot LB and with 1 final submission
# you have no option to risk.

# Improvement should come from:
# Loss function
# Data representation
# Stable CV
# Good features reduction strategy
# Predictions stabilization with NN
# Trend prediction
# Real zero sales detection/classification

# Good kernels references
## (the order is random and the list is not complete):
# https://www.kaggle.com/ragnar123/simple-lgbm-groupkfold-cv
# https://www.kaggle.com/jpmiller/grouping-items-by-stockout-pattern
# https://www.kaggle.com/headsortails/back-to-predict-the-future-interactive-m5-eda
# https://www.kaggle.com/sibmike/m5-out-of-stock-feature
# https://www.kaggle.com/mayer79/m5-forecast-attack-of-the-data-table
# https://www.kaggle.com/yassinealouini/seq2seq
# https://www.kaggle.com/kailex/m5-forecaster-v2
# https://www.kaggle.com/aerdem4/m5-lofo-importance-on-gpu-via-rapids-xgboost

# Features were created in these kernels:
##
# Mean encodings and PCA options
# https://www.kaggle.com/kyakovlev/m5-custom-features
##
# Lags and rolling lags
# https://www.kaggle.com/kyakovlev/m5-lags-features
##
# Base Grid and base features (calendar/price/etc)
# https://www.kaggle.com/kyakovlev/m5-simple-fe

# Personal request
# Please don't upvote any ensemble and copypaste kernels
## The worst case is ensemble without any analyse.
## The best choice - just ignore it.
## I would like to see more kernels with interesting and original approaches.
## Don't feed copypasters with upvotes.

## It doesn't mean that you should not fork and improve others kernels
## but I would like to see params and code tuning based on some CV and analyse
## and not only on LB probing.
## Small changes could be shared in comments and authors can improve their kernel.

## Feel free to criticize this kernel as my knowlege is very limited
## and I can be wrong in code and descriptions.
## Thank you.

In [14]:
final_all_preds.shape

(30490, 29)